# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using GraphLab Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up graphlab create

In [42]:
import graphlab

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [43]:
sales = graphlab.SFrame('kc_house_data.gl/')

# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [44]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [45]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in GraphLab Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [48]:
model_all = graphlab.linear_regression.create(sales, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=1e10,max_iterations=1000)

Linear regression:

--------------------------------------------------------

Number of examples          : 21613

Number of features          : 17

Number of unpacked features : 17

...

| 200       | 201      | 0.000002  | 10.482439    | 7159913.365954     | 367118.702670 |

| 225       | 226      | 0.000002  | 11.572242    | 7159912.787906     | 367118.702668 |

| 250       | 251      | 0.000002  | 12.686854    | 7159912.315042     | 367118.702667 |

+-----------+----------+-----------+--------------+--------------------+---------------+

SUCCESS: Optimal solution found.

Find what features had non-zero weight.

In [50]:
model_all.coefficients.print_rows(num_rows=20)

+------------------+-------+---------------+--------+
|       name       | index |     value     | stderr |
+------------------+-------+---------------+--------+
|   (intercept)    |  None | 540087.585395 |  None  |
|     bedrooms     |  None |      0.0      |  None  |
| bedrooms_square  |  None |      0.0      |  None  |
|    bathrooms     |  None |      0.0      |  None  |
|   sqft_living    |  None |      0.0      |  None  |
| sqft_living_sqrt |  None |      0.0      |  None  |
|     sqft_lot     |  None |      0.0      |  None  |
|  sqft_lot_sqrt   |  None |      0.0      |  None  |
|      floors      |  None |      0.0      |  None  |
|  floors_square   |  None |      0.0      |  None  |
|    waterfront    |  None |      0.0      |  None  |
|       view       |  None |      0.0      |  None  |
|    condition     |  None |      0.0      |  None  |
|      grade       |  None |      0.0      |  None  |
|    sqft_above    |  None |      0.0      |  None  |
|  sqft_basement   |  None |

Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [51]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [55]:
import numpy as np
l1_penalty=np.logspace(1, 7, num=13)
model_all = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, verbose=False,
                                              l2_penalty=0., l1_penalty=31.6227766017,max_iterations=400)
model_all['coefficients']['value'].nnz()

18

In [54]:
def valdationerror(y_predict,y_actual):
    RSS= ((y_actual-y_predict)**2).sum()
    return RSS

for l1_penalty_val in l1_penalty:
    model_all= graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=l1_penalty_val,max_iterations=400,verbose=False)
    pred=model_all.predict(validation)
    print l1_penalty_val,valdationerror(pred,validation['price'])
    

10.0 4.31163506708e+14
31.6227766017 4.3116351098e+14
100.0 4.3116352449e+14
316.227766017 4.31163567215e+14
1000.0 4.31163702353e+14
3162.27766017 4.31164130001e+14
10000.0 4.31165485361e+14
31622.7766017 4.31169710598e+14
100000.0 4.31182902296e+14
316227.766017 4.31221490804e+14
1000000.0 4.31396408621e+14
3162277.66017 4.31900895862e+14
10000000.0 4.35926679856e+14


*** QUIZ QUESTIONS ***
1. What was the best value for the `l1_penalty`?
2. What is the RSS on TEST data of the model with the best `l1_penalty`?

In [56]:
model_all= graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, l2_penalty=0., l1_penalty=10,max_iterations=400,verbose=False)
pred=model_all.predict(training)
print l1_penalty_val,valdationerror(pred,training['price'])
model_all['coefficients']['value'].nnz()

10000000.0 4.65763614754e+14


18

***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

In [29]:
model_all.coefficients.print_rows(num_rows=20)

+------------------+-------+----------------+--------+
|       name       | index |     value      | stderr |
+------------------+-------+----------------+--------+
|   (intercept)    |  None | 607859.203444  |  None  |
|     bedrooms     |  None | -20412.8341352 |  None  |
| bedrooms_square  |  None | 751.949178444  |  None  |
|    bathrooms     |  None | 227.199758292  |  None  |
|   sqft_living    |  None | 270.929850959  |  None  |
| sqft_living_sqrt |  None | -40767.2168554 |  None  |
|     sqft_lot     |  None | 1.26753864319  |  None  |
|  sqft_lot_sqrt   |  None | -1056.99730883 |  None  |
|      floors      |  None | -5135.36788609 |  None  |
|  floors_square   |  None | -2651.13588883 |  None  |
|    waterfront    |  None |  620190.75096  |  None  |
|       view       |  None | 48830.2570235  |  None  |
|    condition     |  None | 63004.1793715  |  None  |
|      grade       |  None |  110226.55142  |  None  |
|    sqft_above    |  None | 335.786470185  |  None  |
|  sqft_ba

# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [30]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [32]:
l1_penalty_values = np.logspace(8, 10, num=20)

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model['coefficients']['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [35]:
def valdationerror(y_predict,y_actual):
    RSS= ((y_actual-y_predict)**2).sum()
    return RSS

for l1_penalty_val in l1_penalty_values:
    model_all= graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=l1_penalty_val,max_iterations=400,verbose=False)
    print l1_penalty_val,'--->',model_all['coefficients']['value'].nnz()
    
#     pred=model_all.predict(validation)
#     print l1_penalty_val,valdationerror(pred,validation['price'])

 100000000.0 ---> 10
127427498.57 ---> 7
162377673.919 ---> 8
206913808.111 ---> 6
263665089.873 ---> 6
335981828.628 ---> 5
428133239.872 ---> 5
545559478.117 ---> 4
695192796.178 ---> 4
885866790.41 ---> 4
1128837891.68 ---> 4
1438449888.29 ---> 4
1832980710.83 ---> 3
2335721469.09 ---> 2
2976351441.63 ---> 1
3792690190.73 ---> 1
4832930238.57 ---> 1
6158482110.66 ---> 1
7847599703.51 ---> 1
10000000000.0 ---> 1


Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzero` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzero` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [39]:
l1_penalty_min = 127427498.57
l1_penalty_max = 162377673.919

***QUIZ QUESTIONS***

What values did you find for `l1_penalty_min` and`l1_penalty_max`? 

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [40]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzero`.

In [41]:
def valdationerror(y_predict,y_actual):
    RSS= ((y_actual-y_predict)**2).sum()
    return RSS

for l1_penalty_val in l1_penalty_values:
    model_all= graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=l1_penalty_val,max_iterations=400,verbose=False)
    
    pred=model_all.predict(validation)
    print l1_penalty_val,valdationerror(pred,validation['price']),model_all['coefficients']['value'].nnz()

127427498.57 5.20468731457e+14 7
129266981.483 5.21170218391e+14 7
131106464.396 5.21878313714e+14 7
132945947.309 5.2259652723e+14 7
134785430.222 5.23325744986e+14 7
136624913.136 5.24068553099e+14 7
138464396.049 5.24825127806e+14 7
140303878.962 5.25599439269e+14 7
142143361.875 5.26391769901e+14 7
143982844.788 5.27199836584e+14 7
145822327.701 5.28027722976e+14 7
147661810.614 5.28875006825e+14 7
149501293.527 5.29740366904e+14 8
151340776.44 5.30621982103e+14 8
153180259.353 5.3152133947e+14 8
155019742.267 5.3243930598e+14 8
156859225.18 5.3337543272e+14 8
158698708.093 5.34329557719e+14 8
160538191.006 5.35302114721e+14 8
162377673.919 5.36290268765e+14 8


***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?

In [57]:
model_all= graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, l2_penalty=0., l1_penalty=127427498.57,max_iterations=400,verbose=False)
pred=model_all.predict(training)
print l1_penalty_val,valdationerror(pred,training['price'])
model_all['coefficients']

10000000.0 5.90818582273e+14


name,index,value,stderr
(intercept),None,-154574.329239,None
bedrooms,None,0.0,None
bedrooms_square,None,0.0,None
bathrooms,None,0.0,None
sqft_living,None,225.570790162,None
sqft_living_sqrt,None,0.0,None
sqft_lot,None,0.0,None
sqft_lot_sqrt,None,-94.4973289171,None
floors,None,0.0,None
floors_square,None,0.0,None
